# Railway Oriented Programming (ROP)

- We don't really want to raise exceptions since it makes the code bloated with error checking
- It's also easy to forget to handle exceptions
- Dependencies might even change the kind of exceptions they throw
- Let's model errors using types instead

In [1]:
class Result:
    pass

In [2]:
class Ok(Result):
    def __init__(self, value):
        self._value = value
    
    def __str__(self):
        return "Ok %s" % str(self._value)

class Error(Result):
    def __init__(self, exn):
        self._exn = exn
    
    def __str__(self):
        return "Error %s" % str(self._exn)

The Expression library contains a similar but more feature complete Result class we can use:

In [3]:
from expression.core import Ok, Error

def fetch(url):
    try:
        if not "http://" in url:
            raise Exception("Error: unable to fetch from: '%s'" % url)
            
        value = url.replace("http://", "")
        return Ok(value)
    except Exception as exn:
        return Error(exn)

In [4]:
result = fetch("http://42")
print(result)

Ok 42


In [5]:
def parse(string):
    try:
        value = float(string)
        return Ok(value)
    except Exception as exn:
        return Error(exn)

In [6]:
result = parse("42")
print(result)

Ok 42.0


# Composition 

How should we compose Result returning functions? How can we make a `fetch_parse` from `fetch` and `parse`. 

Cannot use functional composition since signatures don't match.

```py
def compose(fn: Callable[[A], Result[B, TError]], gn: Callable[[B], Result[C, TError]]) -> Callable[[A], Result[C, TError]]:
    fun x -> ???
```

First we can try to solve this with an "imperative" implementation using type-checks and `if-else` statements:

In [7]:
def fetch_parse(url):
    b = fetch(url)
    if isinstance(b, Ok):
        val_b = b._value # <--- Don't look inside the box!!!
        return parse(val_b)            
    else: # Must be error
        return b

result = fetch_parse("http://42")
print(result)

Ok 42.0


This works, but the code is not easy to read. We have also hard-coded the logic to it's not possible to easily reuse without copy/paste. Here is a nice example on how to solve this by mixing object-oriented code with functional thinking:

In [8]:
class Ok(Result):
    def __init__(self, value):
        self._value = value

    def bind(self, fn):
        return fn(self._value)

    def __str__(self):
        return "Ok %s" % str(self._value)

class Error(Result):
    def __init__(self, exn):
        self._exn = exn
        
    def bind(self, fn):
        return self
    
    def __str__(self):
        return "Error %s" % str(self._exn)
    
def bind(fn, result):
    """We don't want method chaining in Python."""
    return result.bind(fn)

In [9]:
result = bind(parse, fetch("http://42"))
print(result)

Ok 42.0


In [10]:
def compose(f, g):
    return lambda x: f(x).bind(g)

In [11]:
fetch_parse = compose(fetch, parse)
result = fetch_parse("http://123.0")
print(result)

Ok 123.0


In [12]:
result = fetch("http://invalid").bind(parse)
print(result)

Error could not convert string to float: 'invalid'


### But what if we wanted to call fetch 10 times in a row?

This is what's called the "Pyramide of Doom":

In [13]:
from expression.core import result

result = bind(parse, 
            bind(lambda x: fetch("http://%s" % x),
               bind(lambda x: fetch("http://%s" % x),
                  bind(lambda x: fetch("http://%s" % x),
                     bind(lambda x: fetch("http://%s" % x),
                         bind(lambda x: fetch("http://%s" % x),
                             bind(lambda x: fetch("http://%s" % x),
                                 fetch("http://123")
                            )
                         )
                     )
                  )
               )
            )
         )
print(result)

Ok 123.0


## Can we make a more generic compose?

Let's try to make a general compose function that composes two result returning functions:

In [14]:
def compose(f, g):
    return lambda x: f(x).bind(g)

fetch_parse = compose(fetch, parse)
result = fetch_parse("http://42")
print(result)

Ok 42.0


## Pipelining

Functional compose of functions that returns wrapped values is called pipeling in the Expression library. Other languages calls this "Kleisli composition". Using a reducer we can compose any number of functions:

In [15]:
from functools import reduce

def pipeline(*fns):
    return reduce(lambda res, fn: lambda x: res(x).bind(fn), fns)

Now, make `fetch_and_parse` using kleisli:

In [17]:
fetch_and_parse = pipeline(fetch, parse)
result = fetch_and_parse("http://123")
print(result)

Ok 123.0


### What if we wanted to call fetch 10 times in a row?

In [18]:
from expression.extra.result import pipeline

fetch_with_value = lambda x: fetch("http://%s" % x)

request = pipeline(
            fetch,
            fetch_with_value,
            fetch_with_value,
            fetch_with_value,
            fetch_with_value,
            fetch_with_value,
            fetch_with_value,
            parse
          )

result = request("http://123")
print(result)

Ok 123.0
